<h2>FT006 - Inversiones Depositos a la vista </h2>
<p> En este apartado se trata la base de datos FT006 </p>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [2]:
# Estas lineas se utilizan cuando la base de datos está separada por años y se requiere unirla

# # Se carga el FT006 para 2020 y 2021 completo desde el archivo .dta
# FT006_2020_2021 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT006 20202021.dta')
# # Se carga el FT006 para para diciembre de 2019 desde el archivo .dta
# FT006_2019 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT006 122019.dta')
# # Se concatenan la bases de datos 
# frames = [FT006_2020_2021, FT006_2019]
# FT006 = pd.concat(frames, ignore_index=True)
# # Se eliminan la bases que no se necesitan para liberar espacio en la memoria
# del [FT006_2020_2021, FT006_2019, frames]

In [3]:
# Se carga el FT006 completo desde el archivo .csv
FT006 = pd.read_csv('C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Bases_FT/BD_actualizadas/2_Mas_reciente_25082021/FT006.csv', dtype={'Nit':object, 'Año': object, 'Periodo': object, 'Fecha': object}, encoding='latin-1')

In [4]:
# Se genera una nueva columna extrayendo el mes y el año de la variable fecha
FT006['ano'] = FT006['Fecha'].str[:4]
FT006['mes'] = FT006['Fecha'].str[4:6]

In [6]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT006.rename(columns={'Nit': 'nit_eps', 'lineaNegocio': 'linea','claseCuenta': 'clasecuenta', 'saldoExtracto': 'sextracto'}, inplace=True)

In [7]:
# Se transforman los datos para su correcta identificación
FT006[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea']] = FT006[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea']].astype(str)

In [8]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT006 = FT006.loc[FT006['linea'] == '1']

In [9]:
# Se reemplaza la clase de cuenta
FT006.loc[(FT006['clasecuenta'] == 2), 
       'clasecuenta'] = 1 

In [10]:
# Se encuentran las observaciones que cumplen las condiciones. Se deben eliminar las observaciones con clase de cuenta 3
df_filtered = FT006[
    (FT006['clasecuenta'] == 3)
    ].index

In [11]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT006 = FT006.drop(df_filtered)

In [12]:
# Se convierte la variable en mes y año
FT006[['ano', 'mes']] = FT006[['ano', 'mes']].astype(float).astype(int)

In [13]:
# Para hacer la suma de sextracto por clase de cuenta, NIT EPS, NIT proveedor, Mes y Año
FT006 = FT006.groupby(['nit_eps', 'mes', 'ano', 'clasecuenta']).agg({'sextracto': 'sum'}).reset_index()

In [14]:
# Para hacer la suma de sextracto por NIT EPS, NIT proveedor, Mes y Año
FT006 = FT006.groupby(['nit_eps', 'ano', 'mes'])[['sextracto']].sum().reset_index()

<p> Desde aquí se guarda la base de datos con fechas máximas por proveedor </p>

In [18]:
# Se tranforma el formato de las columnas del dataframe 
FT006_FECHA[['ano', 'mes']] = FT006_FECHA[['ano', 'mes']].astype(float).astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT006_FECHA = FT006[
    (FT006['ano'] == 2021)
    ]

In [20]:
# Se procede a eliminar los duplicados dejando el último movimiento registrado. Esto se debe ya que se busca trabajar con el último dato disponible
FT006_FECHA = FT006_FECHA.sort_values(['ano', 'mes']).drop_duplicates('nit_eps', keep='last')

In [21]:
# Para guardar la base de datos lista para ser tratada en formato CSV
FT006_FECHA.to_excel(r'C:/Users/Miguel Angel/OneDrive - Supersalud/Supersalud/Jul 6 Pagos WEB/Pagos WEB/EPS/2021/UNION_BASES/FT006_FECHA.xlsx', index = False)

<p> Hasta aquí se guarda la base de datos con fechas máximas por proveedor </p>

In [22]:
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT006 = FT006.set_index('mes').groupby(['nit_eps', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'ano'], 1)

In [23]:
# Se desagrupa la información para su posterior filtro pues no nos interesa la información para 2019
FT006 = FT006.reset_index()

In [24]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT006[
    (FT006['ano'] == 2019) & (FT006['mes'] == 1) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 2) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 3) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 4) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 5) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 6) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 7) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 8) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 9) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 10) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 11)
    ].index

In [25]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT006 = FT006.drop(df_filtered)

In [26]:
# Se agrupa la información para calcular su respectiva diferencia mensual por CXP
FT006 = FT006.set_index(['nit_eps', 'ano','mes'])

In [27]:
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas
FT006_dif = FT006.diff().fillna(0)

In [28]:
# Se desagregan las bases de datos para su posterior unión
FT006_dif = FT006_dif.reset_index()
FT006 = FT006.reset_index()

In [29]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT006_dif.rename(columns={'sextracto': 'DSaldo'}, inplace=True)

In [30]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT006_dif[
    (FT006['ano'] == 2019)
    ].index

In [31]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT006_dif = FT006_dif.drop(df_filtered)

<p> Desde aquí se filtra la base por la fecha máxima de reporte de la información </p>

In [32]:
# Se seleccionan las columnas requeridas antes del cruce para filtrar la información (se excluye el valor de CXC)
FT006_FECHA = FT006_FECHA[['nit_eps', 'mes', 'ano']]

In [33]:
FT006_dif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1116 entries, 1 to 1161
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   nit_eps  1116 non-null   object 
 1   ano      1116 non-null   int64  
 2   mes      1116 non-null   int64  
 3   DSaldo   1116 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 43.6+ KB


In [34]:
# Se modifican los nombres de las columnas para posterimente pegar las bases de datos
FT006_FECHA.rename(columns={'mes': 'ULTIMO_MES_FT006', 'ano': 'ULTIMO_ANO_FT006'}, inplace=True)

In [35]:
# Se unen las dos bases de datos para obtener la información de FT004_FT003_FT005 y prestadores
FT006 = pd.merge(FT006_dif, FT006_FECHA, how='left', left_on=['nit_eps'], right_on=['nit_eps'])

In [36]:
# Se crea una variable de control para filtrar por fecha del reporte
FT006['periodo'] = pd.to_datetime(FT006.ano*10000 + FT006.mes*100 + 1, format='%Y%m%d')
FT006['periodo_filtro'] = pd.to_datetime(FT006.ULTIMO_ANO_FT006*10000 + FT006.ULTIMO_MES_FT006*100 + 1, format='%Y%m%d')

In [38]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones. La idea es eliminar aquellas observaciones que están por fuera de las fechas máximas de reporte de cada EPS, es decir, si una EPS reportó su información 
df = FT006[
    (FT006['periodo'] <= FT006['periodo_filtro'])
    ]

In [39]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
# FT006_dif.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/DVFICEPS_2020_2021.xlsx', index = False)
FT006_dif.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/DVFICEPS_2020_ene_jul_2021.csv', index = False, encoding='utf-8-sig')

In [40]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT006, FT006_dif, FT006_FECHA, df, df_filtered, fecha_ideal]